In [637]:
import os
import json
import pandas as pd
import numpy as np
from pprint import pprint
import warnings 
warnings.filterwarnings('ignore')

In [13]:
params = {}
params['LIBRARY_NAME'] = 'sklearn'
params['LIBRARY_OBJECT_NAME'] = 'ensemble' #'linear_model'
params['LIBRARY_FUNCTION_NAME'] = 'RandomForestRegressor'
params

{'LIBRARY_FUNCTION_NAME': 'RandomForestRegressor',
 'LIBRARY_NAME': 'sklearn',
 'LIBRARY_OBJECT_NAME': 'ensemble'}

In [14]:
def get_estimator(params):
    module_ = __import__(params['LIBRARY_NAME']+'.'+params['LIBRARY_OBJECT_NAME'])
    class_ = getattr(module_, params['LIBRARY_OBJECT_NAME'])
    clf_ = getattr(class_, params['LIBRARY_FUNCTION_NAME'])
    clf = clf_()
    return clf  

In [15]:
get_clf = get_estimator(params)
get_clf

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [12]:
from sklearn.preprocessing import OneHotEncoder
OneHotEncoder()

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [1]:
params = {}
params['LIBRARY_NAME'] = 'sklearn'
params['LIBRARY_OBJECT_NAME'] = 'preprocessing' #'linear_model'
params['LIBRARY_FUNCTION_NAME'] = 'MinMaxScaler'
params

{'LIBRARY_FUNCTION_NAME': 'MinMaxScaler',
 'LIBRARY_NAME': 'sklearn',
 'LIBRARY_OBJECT_NAME': 'preprocessing'}

In [8]:
def get_transformer(params):
    module_ = __import__(params['LIBRARY_NAME']+'.'+params['LIBRARY_OBJECT_NAME'])
    class_ = getattr(module_, params['LIBRARY_OBJECT_NAME'])
    transformer_ = getattr(class_, params['LIBRARY_FUNCTION_NAME'])
    transformer  = transformer_()
    return transformer   


In [9]:
get_clf = get_transformer(params)
get_clf

MinMaxScaler(copy=True, feature_range=(0, 1))

### **train_data_transformer(2019.09.03)**

In [269]:
current_dir=os.getcwd()
data_dir = os.path.join(os.path.join(current_dir, 'example'),'bike_demand')
data_dir
bike = pd.read_csv(data_dir+'\\bike_train.csv')
bike.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [28]:
from sklearn.preprocessing import StandardScaler
sd_scaler = StandardScaler(copy=True, with_mean=True, with_std=True)

from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler(copy=True, feature_range=(0, 1))

from sklearn.preprocessing import OneHotEncoder
oh_encode = OneHotEncoder(categorical_features=None, 
                          categories=None,
                          #dtype='numpy.float64', 
                          handle_unknown='error',
                          n_values=None, 
                          sparse=True)

In [5]:
DATA = bike.copy() 
field_name = 'temp'
field_column = bike['temp']
transformer = mm_scaler

In [674]:
params = {}
params['field_name'] = "temp, humidity"
params['transformer'] = mm_scaler
request = dict(req01=params)

params = {}
params['field_name'] = 'atemp'
params['transformer'] = sd_scaler
request['req02'] = params

params = {}
params['field_name'] = 'weather'
params['transformer'] = oh_encode
request['req03'] = params

params = {}
params['field_name'] = 'season'
params['transformer'] = le_encode
request['req04'] = params


request

{'req01': {'field_name': 'temp, humidity',
  'transformer': MinMaxScaler(copy=True, feature_range=(0, 1))},
 'req02': {'field_name': 'atemp',
  'transformer': StandardScaler(copy=True, with_mean=True, with_std=True)},
 'req03': {'field_name': 'weather',
  'transformer': OneHotEncoder(categorical_features=None, categories=None,
         dtype=<class 'numpy.float64'>, handle_unknown='error',
         n_values=None, sparse=True)},
 'req04': {'field_name': 'season', 'transformer': LabelEncoder()}}

In [686]:
def train_data_transformer(data, field_name, field_column, transformer): #학습용으로 사용될 데이터에 전처리를 수행하고 결과를 리턴하는 함수
    print('{}필드에 {}전처리를 수행합니다.'.format(field_name, type(transformer).__name__))
    #field_column = field_column.astype(float)
    
    #변환기를 학습데이터로 학습하고(fit) 변환하는(transform) 과정
    try: #필드가 두 개 이상일 때 
        field_column.shape[1]
        transformer.fit(field_column)
        changed_field = transformer.transform(field_column)
    except: #필드가 하나일 때 
        transformer.fit(field_column.values.reshape(-1,1)) 
        changed_field = transformer.transform(field_column.values.reshape(-1,1))
    
    #변환기 정보를 딕셔너리로 저장하는 과정
    transformer_information = dict(encoder_name = type(transformer).__name__)
    
    #transfrom된 데이터와 원본 데이터 통합
    if type(transformer).__name__ == 'OneHotEncoder': #원핫인코딩을 한 경우 통합 방법(새로운 칼럼 생성됨)
        changed_field = changed_field.toarray() if not isinstance(changed_field, np.ndarray) else changed_field
        OneHot = pd.DataFrame(changed_field, columns = [field_name+"_"+str(int(i)) for i in range(changed_field.shape[1])])
        data = pd.concat([data, OneHot], axis=1, sort=False)
        data = data.drop(field_name, axis=1)
        transformer_information['orginal_classes'] = list(list(transformer.categories_[0]))
        transformer_information['encoded_classes'] = transformer.transform(np.array(transformer.categories_).reshape(4,1)).toarray().tolist(),
        
        return data, transformer_information
    else: #원핫인코딩이 아닌 경우 통합 방법(기존 값을 덮어씀)
        data[field_name] = changed_field
        if type(transformer).__name__ == 'LabelEncoder':
            transformer_information['orginal_classes'] = list(transformer.classes_)
            transformer_information['encoded_classes'] = list(np.unique(changed_field))
        return data, transformer_information

In [692]:
DATA = bike.copy() 
DATA.head(1)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16


In [693]:
def season_as_string(x):
    if x == 1:
        return '봄'
    elif x == 2:
        return '여름'
    elif x == 3:
        return '가을'
    elif x == 4:
        return '겨울'
    
DATA['season']=list(map(season_as_string, DATA['season']))
DATA.head(1)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,봄,0,0,1,9.84,14.395,81,0.0,3,13,16


In [695]:
preprocessed_data_info = {}

for user_req in request:
    user_request = request[user_req]
    field_name = user_request['field_name']
    print(field_name)
    if ',' in field_name:
        field_names = field_name.split(',')
        for field_name_ in field_names:
            field_name_ = field_name_.strip()
            field_column = DATA[field_name_]
            transformer = user_request['transformer']
            DATA,  info = train_data_transformer(data=DATA, field_name=field_name_, field_column=field_column, transformer=transformer)
            preprocessed_data_info[field_name_] = info
    else:
        field_name = user_request['field_name']
        field_column = DATA[user_request['field_name']]
        transformer = user_request['transformer']
        DATA,  info = train_data_transformer(data=DATA, field_name=field_name, field_column=field_column, transformer=transformer)
        preprocessed_data_info[field_name] = info

#DATA.head()
preprocessed_data_info

temp, humidity
temp필드에 MinMaxScaler전처리를 수행합니다.
humidity필드에 MinMaxScaler전처리를 수행합니다.
atemp
atemp필드에 StandardScaler전처리를 수행합니다.
weather
weather필드에 OneHotEncoder전처리를 수행합니다.
season
season필드에 LabelEncoder전처리를 수행합니다.


{'atemp': {'encoder_name': 'StandardScaler'},
 'humidity': {'encoder_name': 'MinMaxScaler'},
 'season': {'encoded_classes': [0, 1, 2, 3],
  'encoder_name': 'LabelEncoder',
  'orginal_classes': ['가을', '겨울', '봄', '여름']},
 'temp': {'encoder_name': 'MinMaxScaler'},
 'weather': {'encoded_classes': ([[1.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]],),
  'encoder_name': 'OneHotEncoder',
  'orginal_classes': [1, 2, 3, 4]}}

In [356]:
data['season']=list(map(season_as_string, data['season']))

In [391]:
data['season'].unique()

array(['봄', '여름', '가을', '겨울'], dtype=object)

In [403]:
from sklearn.preprocessing import LabelEncoder
print(data['season'].unique())
le_encode = LabelEncoder()
le_encode.fit(data['season'])
transformed_data = le_encode.transform(data['season'])

encode_information = dict(인코딩이전변수값 = list(le_encode.classes_),
                          인코딩이후변수값 = list(np.unique(transformed_data)),
                          encoder_name = type(le_encode).__name__, )
data['season'] = transformed_data
print(data['season'].unique())
encode_information

['봄' '여름' '가을' '겨울']
[2 3 0 1]


{'encoder_name': 'LabelEncoder',
 '인코딩이전변수값': ['가을', '겨울', '봄', '여름'],
 '인코딩이후변수값': [0, 1, 2, 3]}

In [589]:
DATA = bike.copy()
DATA['season']=list(map(season_as_string, DATA['season']))
DATA.head(1)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,봄,0,0,1,9.84,14.395,81,0.0,3,13,16


In [590]:
from sklearn.preprocessing import OneHotEncoder
print(DATA['season'].unique())
oh_encode = OneHotEncoder()
oh_encode.fit(DATA['season'].values.reshape(-1,1))
transformed_data = oh_encode.transform(DATA['season'].values.reshape(-1,1))
transformed_data = transformed_data.toarray()

encode_information = dict(인코딩이전변수값 = list(list(oh_encode.categories_[0])),
                          인코딩이후변수값 = oh_encode.transform(np.array(oh_encode.categories_).reshape(4,1)).toarray().tolist(),
                          encoder_name = type(oh_encode).__name__, )


OneHot = pd.DataFrame(transformed_data, columns = ['season'+"_"+str(int(i)) for i in range(transformed_data.shape[1])])
DATA = pd.concat([DATA, OneHot], axis=1, sort=False)
DATA = DATA.drop('season', axis=1)

encode_information

['봄' '여름' '가을' '겨울']


{'encoder_name': 'OneHotEncoder',
 '인코딩이전변수값': ['가을', '겨울', '봄', '여름'],
 '인코딩이후변수값': [[1.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 1.0]]}

In [592]:
from sklearn.preprocessing import OneHotEncoder
print(DATA['weather'].unique())
oh_encode = OneHotEncoder()
oh_encode.fit(DATA['weather'].values.reshape(-1,1))
transformed_data = oh_encode.transform(DATA['weather'].values.reshape(-1,1))
transformed_data = transformed_data.toarray()

encode_information = dict(인코딩이전변수값 = list(list(oh_encode.categories_[0])),
                          인코딩이후변수값 = oh_encode.transform(np.array(oh_encode.categories_).reshape(4,1)).toarray().tolist(),
                          encoder_name = type(oh_encode).__name__, )


OneHot = pd.DataFrame(transformed_data, columns = ['weather'+"_"+str(int(i)) for i in range(transformed_data.shape[1])])
DATA = pd.concat([DATA, OneHot], axis=1, sort=False)
DATA = DATA.drop('weather', axis=1)

encode_information

[1 2 3 4]


c:\users\daumsoft\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


{'encoder_name': 'OneHotEncoder',
 '인코딩이전변수값': [1.0, 2.0, 3.0, 4.0],
 '인코딩이후변수값': [[1.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 1.0]]}

In [339]:
#data['season'] = transformed_data
print(data['season'].unique())
data.head()

[2 3 0 1]


,datetime,season,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,weather_0,weather_1,weather_2,weather_3
0,2011-01-01 00:00:00,2,0,0,0.224490,-1.092737,0.81,0.0,3,13,16,1.0,0.0,0.0,0.0
1,2011-01-01 01:00:00,2,0,0,0.204082,-1.182421,0.80,0.0,8,32,40,1.0,0.0,0.0,0.0
2,2011-01-01 02:00:00,2,0,0,0.204082,-1.182421,0.80,0.0,5,27,32,1.0,0.0,0.0,0.0
3,2011-01-01 03:00:00,2,0,0,0.224490,-1.092737,0.75,0.0,3,10,13,1.0,0.0,0.0,0.0
4,2011-01-01 04:00:00,2,0,0,0.224490,-1.092737,0.75,0.0,0,1,1,1.0,0.0,0.0,0.0


In [350]:
data = train_data_transformer(data=data, field_name='season', field_column=data['season'],
                              transformer=le_encode)
data.head()

season필드에 LabelEncoder전처리를 수행합니다.


c:\users\daumsoft\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\daumsoft\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,datetime,season,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,weather_0,weather_1,weather_2,weather_3
0,2011-01-01 00:00:00,2,0,0,0.224490,-1.092737,0.81,0.0,3,13,16,1.0,0.0,0.0,0.0
1,2011-01-01 01:00:00,2,0,0,0.204082,-1.182421,0.80,0.0,8,32,40,1.0,0.0,0.0,0.0
2,2011-01-01 02:00:00,2,0,0,0.204082,-1.182421,0.80,0.0,5,27,32,1.0,0.0,0.0,0.0
3,2011-01-01 03:00:00,2,0,0,0.224490,-1.092737,0.75,0.0,3,10,13,1.0,0.0,0.0,0.0
4,2011-01-01 04:00:00,2,0,0,0.224490,-1.092737,0.75,0.0,0,1,1,1.0,0.0,0.0,0.0


In [352]:
data = train_data_transformer(data=data, field_name='workingday', field_column=data['workingday'],
                              transformer=oh_encode)
data.head()

workingday필드에 OneHotEncoder전처리를 수행합니다.


,datetime,holiday,temp,atemp,humidity,windspeed,casual,registered,count,weather_0,weather_1,weather_2,weather_3,season_0,season_1,season_2,season_3,workingday_0,workingday_1
0,2011-01-01 00:00:00,0,0.224490,-1.092737,0.81,0.0,3,13,16,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2011-01-01 01:00:00,0,0.204082,-1.182421,0.80,0.0,8,32,40,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2011-01-01 02:00:00,0,0.204082,-1.182421,0.80,0.0,5,27,32,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2011-01-01 03:00:00,0,0.224490,-1.092737,0.75,0.0,3,10,13,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,2011-01-01 04:00:00,0,0.224490,-1.092737,0.75,0.0,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [290]:
DATA=bike.copy()
mm_scaler.fit(DATA['humidity'].values.reshape(-1,1))
n_data = mm_scaler.transform(DATA['humidity'].values.reshape(-1,1))
DATA['humidity'] = n_data
DATA.head()

c:\users\daumsoft\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,0.81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,0.80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,0.80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,0.75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,0.75,0.0,0,1,1


In [291]:
DATA=bike.copy()
mm_scaler.fit(DATA[['temp', 'humidity']])
n_data = mm_scaler.transform(DATA[['temp', 'humidity']])
DATA[['temp', 'humidity']] = n_data
DATA.head()

c:\users\daumsoft\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,0.224490,14.395,0.81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,0.204082,13.635,0.80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,0.204082,13.635,0.80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,0.224490,14.395,0.75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,0.224490,14.395,0.75,0.0,0,1,1


In [183]:
def train_data_transformer(data, field_name, field_column, transformer): #학습용으로 사용될 데이터에 전처리를 수행하고 결과를 리턴하는 함수
    try:
        transformer.fit(field_column) #필드가 두 개 이상일 때 
        changed_field = transformer.transform(field_column)
    except:
        transformer.fit(field_column.values.reshape(-1,1)) #필드가 하나일 때 
        changed_field = transformer.transform(field_column.values.reshape(-1,1))
        
    if changed_field.shape[1] == 1: #numerical 전처리의 경우에 해당 
        
        print('12', changed_field.shape[1])
        data[field_name] = changed_field
        return data
        
    else: #그 외 (예, 원핫인코딩)에 해당
        print('34', changed_field.shape[1])
        changed_field = changed_field.toarray() if not isinstance(changed_field, np.ndarray) else changed_field
        OneHot = pd.DataFrame(changed_field, columns = ["season_"+str(int(i)) for i in range(changed_field.shape[1])])
        data = pd.concat([data, OneHot], axis=1, sort=False).drop(field_name, axis=1)
        return data

In [184]:
DATA = bike.copy() 
data = train_data_transformer(data=DATA, field_name='season', field_column=DATA['season'], transformer=oh_encode)
data.head()

34 4


,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,season_0,season_1,season_2,season_3
0,2011-01-01 00:00:00,0,0,1,9.84,14.395,81,0.0,3,13,16,1.0,0.0,0.0,0.0
1,2011-01-01 01:00:00,0,0,1,9.02,13.635,80,0.0,8,32,40,1.0,0.0,0.0,0.0
2,2011-01-01 02:00:00,0,0,1,9.02,13.635,80,0.0,5,27,32,1.0,0.0,0.0,0.0
3,2011-01-01 03:00:00,0,0,1,9.84,14.395,75,0.0,3,10,13,1.0,0.0,0.0,0.0
4,2011-01-01 04:00:00,0,0,1,9.84,14.395,75,0.0,0,1,1,1.0,0.0,0.0,0.0
